# 第4章：TodoList任务管理 - 终极优雅版

## 本章目标

本章将实现 Claude Code 最亮眼的功能：**任务清单自动管理**。你将学习：

- **扩展State定义**: 添加自定义字段 `todo_list` 到 AgentState
- **InjectedState机制**: 如何将状态注入到tool这种
- **ToolNode的威力**: 为什么直接使用 ToolNode 是最佳实践
- **Command返回值**: todo_write 如何通过 Command 更新状态
- **提示词工程**: 80% 的功能来自精心设计的 Prompt
- **任务自动分解**: LLM 如何自主生成任务清单
- **完整数据流**: 从 LLM 推理到 State 更新的全过程

## 应用场景

- **复杂项目开发**: 自动分解和跟踪开发任务
- **多步骤操作**: 清晰展示当前进度
- **用户体验**: 让用户了解 Agent 在做什么
- **可观测性**: 记录任务执行历史

## 💡 这是 Claude Code 的什么功能？

这是 Claude Code 最亮眼的功能：**任务清单管理**。当你给一个复杂任务时，它会自动分解并显示进度：

```
Tasks:
☑ Analyzing project structure
☑ Installing dependencies  
⚙ Converting files to TypeScript (in_progress)
☐ Fixing type errors
☐ Running tests
```

**关键价值**：
- **可见性**：你清楚知道 AI 在做什么，进度如何
- **可靠性**：AI 不会遗漏子任务，确保完整完成
- **可中断**：可以随时停止，AI 知道下次从哪里继续

**技术秘密（80% 是 Prompt Engineering！）**：
- State 中添加 `todo_list` 字段存储状态
- 通过精心设计的 prompt 指导 LLM 自主生成更新todo_list的**工具调用**
- 通过todo_write工具将LLM生成的任务清单更新到state
- 提示词引导LLM频繁调用todo_read工具获取任务清单

详细过程可参考第11章的[完整数据流图解](#sec-purpose)



## 1. 环境设置和导入

In [ ]:
import os
import uuid
from datetime import datetime
from typing import Annotated, List, Optional
from enum import Enum

# LangGraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import InjectedState, ToolNode  # 关键导入！
from langgraph.types import Command
from pydantic import BaseModel, Field

# LangChain
from langchain_core.tools import tool, InjectedToolCallId
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage, ToolMessage

# LLM (选择一个)
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi

print("✅ 导入完成")

In [ ]:
# 配置环境变量
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "chapter-04-todo-task"

# 初始化 LLM
llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
# llm = ChatTongyi(model="qwen3-max", temperature=0)

print(f"✅ LLM 初始化成功: {llm.model_name}")

## 2. 数据结构定义

In [ ]:
class TaskStatus(str, Enum):
    """任务状态枚举"""
    PENDING = "pending"           # 待执行
    IN_PROGRESS = "in_progress"   # 执行中
    COMPLETED = "completed"       # 已完成
    FAILED = "failed"             # 失败


class TodoItem(BaseModel):
    """任务项 - 使用 Pydantic 模型
    
    关键优势：
    - 自动类型验证
    - 自动生成默认值
    - LLM 可以直接生成符合此模型的数据
    """
    # LLM 如果忘记提供 ID，default_factory 会自动创建一个
    id: str = Field(default_factory=lambda: str(uuid.uuid4())[:8])
    
    # 必填字段
    name: str
    
    # 字段默认值
    desc: str = ""
    status: TaskStatus = TaskStatus.PENDING  # 自动默认为 "pending"
    
    # 可选字段
    start_time: Optional[str] = None
    end_time: Optional[str] = None
    error: Optional[str] = None


class AgentState(BaseModel):
    """Agent 状态 - 使用 Pydantic 模型
    
    关键优势：
    - 在 Pydantic 中使用 LangGraph 的 Reducer
    - 语法是 "Annotated[TYPE, REDUCER] = Field(default_factory=...)"
    """
    messages: Annotated[List[BaseMessage], add_messages] = Field(default_factory=list)
    todo_list: List[TodoItem] = Field(default_factory=list)


print("✅ 数据结构定义完成")
print(f"状态类型: {[s.value for s in TaskStatus]}")

## 3. 辅助函数

这些函数帮助格式化和验证任务数据。

In [ ]:
def format_todo_list(todo_list: List[TodoItem]) -> str:
    """将 todo_list 格式化为美观的、人类可读的字符串
    
    - 仅显示 id, name, desc, status
    - 按 status 分组
    """
    if not todo_list:
        return "✅ 任务列表为空。"

    # 1. 按状态分组
    by_status = {
        "in_progress": [],
        "pending": [],
        "completed": [],
        "failed": []
    }
    
    for task in todo_list:
        status = task.status
        if status not in by_status:
            status = "pending"
        by_status[status].append(task)

    # 2. 定义状态标题和表情
    status_headers = {
        "in_progress": "🔄 进行中 (In-Progress):",
        "pending": "⏳ 待处理 (Pending):",
        "completed": "✅ 已完成 (Completed):",
        "failed": "❌ 失败 (Failed):"
    }
    
    result_lines = ["\n" + "=" * 30 + " 任务列表 " + "=" * 30]
    has_content = False

    # 3. 按期望的顺序构建输出
    for status, header in status_headers.items():
        tasks = by_status[status]
        
        if tasks:
            has_content = True
            result_lines.append(f"\n{header}")
            for task in tasks:
                result_lines.append(f"  [{task.id}] {task.name}")
                result_lines.append(f"      描述: {task.desc}")
            
    result_lines.append("\n" + "=" * 62)

    if not has_content:
        return "✅ 任务列表为空。"
        
    return "\n".join(result_lines)


def _validate_todo_list(todo_list: List[TodoItem]) -> List[TodoItem]:
    """验证和处理 Pydantic 任务列表
    
    Pydantic 已经完成了：
    1. 类型检查 (例如, name 必须是 str)
    2. 默认值 (id, status, desc)
    
    我们只需要处理业务逻辑，比如根据状态设置时间戳。
    """
    current_time = datetime.now().isoformat()
    
    # Pydantic 模型是可变的，我们可以直接修改它们
    for task in todo_list:
        # 业务逻辑：设置开始/结束时间戳
        if task.status == TaskStatus.IN_PROGRESS and not task.start_time:
            task.start_time = current_time
            
        if task.status in [TaskStatus.COMPLETED, TaskStatus.FAILED] and not task.end_time:
            task.end_time = current_time

    return todo_list


print("✅ 辅助函数定义完成")

## 4. 核心概念：InjectedState 和 ToolNode

在深入实现之前，让我们理解两个关键概念：

### 🔑 InjectedState - 自动状态注入

**问题**：工具函数需要访问 State，但 LLM 无法传递整个 State（太大了）

**解决方案**：使用 `InjectedState` 注解，让 ToolNode 自动注入

```python
@tool
def my_tool(
    business_data: str,  # LLM 传递
    state: Annotated[AgentState, InjectedState]  # ToolNode 自动注入
) -> str:
    # 可以访问 state["todo_list"] 等字段
    return "result"
```

**LLM 的 tool_call**：
```json
{
  "name": "my_tool",
  "arguments": {
    "business_data": "value"  
    // 注意：没有 state 参数！
  }
}
```

### ✅ ToolNode - 预构建的工具节点

`ToolNode` 是 LangGraph 的预构建节点，自动处理：

1. 解析 LLM 的 tool_calls
2. 提取参数
3. **注入 InjectedState 参数**
4. 调用工具函数
5. **捕获 State 修改**（通过 Command 返回值）
6. 构造正确的返回值

**使用方式**：
```python
from langgraph.prebuilt import ToolNode

# 一行搞定！
tool_node = ToolNode([todo_read, todo_write, ...])

# 添加到图中
builder.add_node("tools", tool_node)
```

### 📊 Command 返回值 - 更新 State

工具函数可以通过返回 `Command` 对象来更新 State：

```python
from langgraph.types import Command

@tool
def todo_write(
    todo_list: List[TodoItem],
    state: Annotated[AgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
):
    # 验证数据
    validated = _validate_todo_list(todo_list)
    
    # 返回 Command 更新 State
    return Command(
        update={
            "todo_list": validated,  # 更新 todo_list 字段
            "messages": [ToolMessage("更新成功", tool_call_id=tool_call_id)]
        }
    )
```

**ToolNode 会自动处理这个 Command**，将其转换为正确的 State 更新！

## 5. TodoRead 工具实现

读取当前任务列表，返回格式化的字符串。

In [ ]:
@tool
def todo_read(state: Annotated[AgentState, InjectedState]) -> str:
    """读取当前会话的任务列表

    主动且频繁地使用此工具，以确保你了解当前任务列表的状态。
    你应该尽可能多地使用此工具，特别是在：
    - 开始工作之前
    - 完成任务后
    - 不确定下一步做什么时

    Args:
        state: Agent 状态（通过 InjectedState 自动注入）

    Returns:
        格式化的任务列表
    """
    todo_list = state.todo_list

    if not todo_list:
        return "任务列表为空。如果用户给了复杂任务，请使用 todo_write 创建任务列表。"

    # 格式化输出
    result = ["当前任务列表:\n"]
    by_status = {"in_progress": [], "pending": [], "completed": [], "failed": []}
    for task in todo_list:
        status = task.status
        by_status[status].append(task)
    
    if by_status["in_progress"]:
        result.append("🔄 进行中:")
        for task in by_status["in_progress"]:
            result.append(f"  [{task.id}] {task.name}")
        result.append("")
    if by_status["pending"]:
        result.append("⏳ 待处理:")
        for task in by_status["pending"]:
            result.append(f"  [{task.id}] {task.name}")
        result.append("")
    if by_status["completed"]:
        result.append("✅ 已完成:")
        for task in by_status["completed"]:
            result.append(f"  [{task.id}] {task.name}")
        result.append("")
    if by_status["failed"]:
        result.append("❌ 失败:")
        for task in by_status["failed"]:
            result.append(f"  [{task.id}] {task.name}")
        result.append("")
    result.append(f"\n总计: {len(todo_list)} 个任务")

    return "\n".join(result)


print("✅ TodoRead 工具创建完成")

## 6. TodoWrite 工具实现

这是核心工具！LLM 通过它来创建和更新任务列表。

### 🔑 关键设计要点

1. **返回 Command 对象**：通过 `Command(update={...})` 更新 State
2. **使用 InjectedToolCallId**：获取工具调用 ID（用于 ToolMessage）
3. **验证业务逻辑**：检查并发限制、设置时间戳
4. **打印调试信息**：显示更新前后的任务列表（仅用于演示）

In [ ]:
@tool
def todo_write(
    todo_list: List[TodoItem],
    state: Annotated[AgentState, InjectedState], # 这里仅用于对比更新前和更新后的状态
    tool_call_id: Annotated[str, InjectedToolCallId]
):
    """更新当前会话的任务列表

    主动使用此工具来跟踪进度和管理任务执行。

    ## 何时使用此工具
    在以下场景中主动使用此工具：
    1. 收到复杂的多步骤任务时 - 立即分解为子任务
    2. 开始执行任务时 - 将任务标记为 in_progress
    3. 完成任务后 - 将任务标记为 completed
    4. 遇到错误时 - 将任务标记为 failed 并记录错误

    ## 任务状态管理
    1. **任务状态**: 使用这些状态来跟踪进度：
       - pending: 任务尚未开始
       - in_progress: 当前正在执行（同一时间最多3个）
       - completed: 任务成功完成
       - failed: 任务遇到错误

    2. **任务管理规则**:
       - 实时更新任务状态
       - 同一时间最多3个任务处于 in_progress
       - 必须按顺序处理任务
       - 任务失败时，标记为 failed 并包含错误详情

    3. **任务完成要求**:
       - 只有在完全完成时才标记为 completed
       - 如果遇到错误，标记为 failed
       - 绝不要在以下情况标记为 completed：
         * 实现不完整
         * 遇到未解决的错误
         * 找不到必要的文件或依赖

    Args:
        todo_list: 更新后的完整任务列表
        state: Agent 状态，包含更新前的todo_list
        tool_call_id: 本次工具调用对应的id（通过InjectedToolCallId注入）
    """
    
    # 1. 获取更新前的任务列表（用于打印对比）
    old_todo_list = state.todo_list
    formated_old = format_todo_list(old_todo_list)
    print(f"更新前todo_list：\n{formated_old}")
    
    # 2. 验证和处理任务
    validated_tasks = _validate_todo_list(todo_list)
    formated_new = format_todo_list(validated_tasks)
    print(f"更新后todo_list：\n{formated_new}")

    # 3. 检查并发任务限制
    in_progress_count = sum(1 for t in validated_tasks if t.status == "in_progress")
    if in_progress_count > 3:
        return Command(
            update={
                "messages": [ToolMessage(
                    f"错误: 同时进行的任务数 ({in_progress_count}) 超过限制 (3)。", 
                    tool_call_id=tool_call_id
                )]
            }
        )

    # 4. 生成反馈信息
    summary = "任务列表已更新，使用todo_read查看任务状态"
    
    # 5. 返回 Command 更新 State
    return Command(
        update={
            "todo_list": validated_tasks,  # 更新 todo_list 字段
            "messages": [ToolMessage(summary, tool_call_id=tool_call_id)]
        }
    )


print("✅ TodoWrite 工具创建完成")

## 7. 辅助工具（用于演示）

In [ ]:
@tool
def create_file(filename: str, content: str) -> str:
    """创建文件（模拟）"""
    return f"✅ 已创建文件 {filename}，内容长度: {len(content)} 字符"


@tool
def run_tests() -> str:
    """运行测试（模拟）"""
    return "✅ 测试运行完成：10个测试全部通过"


# 所有工具
tools = [todo_read, todo_write, create_file, run_tests]

print(f"✅ 创建了 {len(tools)} 个工具")

## 8. 系统提示词设计

### 🎯 提示词工程的核心价值

80% 的功能来自这个提示词！它教会 LLM：

1. **何时创建任务列表**：3+步骤、复杂任务
2. **如何分解任务**：通过示例教学
3. **任务状态转换**：pending → in_progress → completed/failed
4. **管理原则**：频繁读取、实时更新、按顺序执行

### 📝 提示词设计要点

- ✅ 使用明确指令："你必须使用 todo_write"
- ✅ 提供具体场景："当任务需要3个或更多步骤时"
- ✅ 包含 Few-shot 示例：演示完整流程
- ✅ 强调关键规则：频繁使用、实时更新

In [ ]:
SYSTEM_PROMPT = """你是一个高效的AI助手，具有强大的任务管理能力。

## 任务管理规则

### 何时创建任务列表
当用户请求满足以下条件时，你必须使用 todo_write 创建任务列表：
1. 任务需要3个或更多步骤
2. 任务复杂且需要仔细规划
3. 用户明确要求使用任务列表
4. 用户提供了多个待办事项

### 任务执行流程
1. 收到复杂任务 → 使用 todo_write 创建任务列表
2. 开始执行任务 → 使用 todo_read 查看任务，然后用 todo_write 标记为 in_progress
3. 完成任务 → 使用 todo_write 标记为 completed
4. 遇到错误 → 使用 todo_write 标记为 failed，包含错误信息

### 任务状态转换
pending → in_progress → completed/failed

### 重要原则
- 频繁使用 todo_read 检查当前状态
- 实时更新任务状态
- 同时最多3个 in_progress 任务
- 按顺序执行任务
- 向用户清晰报告进度

## 示例

用户: "创建一个Python项目，包含主文件、测试文件，并运行测试"

你的行动:
1. 使用 todo_write 创建任务列表:
   - 创建主文件 main.py
   - 创建测试文件 test_main.py
   - 运行测试
2. 执行第一个任务前，用 todo_write 标记为 in_progress
3. 完成后，用 todo_write 标记为 completed
4. 继续下一个任务...
"""

print("✅ 系统提示词定义完成")

## 9. 节点定义

In [ ]:
def agent_node(state: AgentState, llm_with_tools):
    """Agent 节点：调用 LLM 生成响应"""
    messages = [SystemMessage(content=SYSTEM_PROMPT)] + state.messages
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}


def should_continue(state: AgentState) -> str:
    """条件边：判断是否继续执行工具"""
    last_message = state.messages[-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END


print("✅ 节点函数定义完成")

## 10. 构建图

### ✨ 关键：直接使用 ToolNode

`ToolNode` 会自动处理：
- InjectedState 注入
- Command 返回值处理
- State 更新

**不需要任何包装器！**

In [ ]:
def build_graph():
    """构建 StateGraph"""
    
    # 准备 LLM
    llm_with_tools = llm.bind_tools(tools)
    
    # ✅ 关键：直接使用预构建的 ToolNode
    # 它会自动处理：
    # 1. InjectedState 注入
    # 2. Command 返回值处理
    # 3. State 更新
    tool_node = ToolNode(tools)
    
    # 创建 StateGraph
    builder = StateGraph(AgentState)
    
    # 添加节点
    builder.add_node("agent", lambda state: agent_node(state, llm_with_tools))
    builder.add_node("tools", tool_node)  # ✅ 直接使用 tool_node
    
    # 添加边
    builder.add_edge(START, "agent")
    builder.add_conditional_edges("agent", should_continue, {
        "tools": "tools",
        END: END
    })
    builder.add_edge("tools", "agent")
    
    # 编译
    return builder.compile()


# 构建图
print("正在构建 LangGraph...")
graph = build_graph()
print("✅ 图构建完成！")

## 11. 完整数据流图解 <a id="sec-purpose" style="border:none;"></a>

让我们通过一个完整的例子理解数据流：

### 场景：用户说 "创建3个文件：main.py、test.py、config.py"

#### 第1轮：LLM 决定创建任务列表

```
┌─────────────────────────────────────────────────────────────┐
│  ① 用户输入进入 Agent 节点                                   │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ② Agent节点：LLM推理 ⭐ 任务清单生成点                     │
│                                                              │
│  系统提示词触发：                                            │
│    - "任务需要3个或更多步骤" ✓                               │
│                                                              │
│  LLM自主分解：                                               │
│    用户要创建3个文件                                         │
│    → 分解为3个独立的创建任务                                 │
│    → 每个任务设置为 pending 状态                            │
│                                                              │
│  生成 tool_call：                                            │
│    {                                                         │
│      "name": "todo_write",                                   │
│      "arguments": {                                          │
│        "todo_list": [                                        │
│          {"name": "创建main.py", "status": "pending"},      │
│          {"name": "创建test.py", "status": "pending"},      │
│          {"name": "创建config.py", "status": "pending"}     │
│        ]                                                     │
│      }                                                       │
│    }                                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ③ should_continue：检测到 tool_calls，路由到 tools 节点      │
└─────────────────────────────────────────────────────────────┘
```

#### 第2轮：执行 todo_write 工具

```
┌─────────────────────────────────────────────────────────────┐
│  ① ToolNode 执行 todo_write                                 │
│                                                             │
│  ② ToolNode 解析 tool_calls                                  |
│                                                             │
│  ③ 调用 todo_write(                                         │
│       todo_list=[...],  # LLM传的                            │
│       state=current_state,  # ToolNode注入的,可忽略           │
│       tool_call_id="xyz"  # ToolNode注入的                   │
│     )                                                       │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ④ todo_write 执行并返回 Command                            │
│                                                              │
│  def todo_write(todo_list, state, tool_call_id):            │
│      # 验证任务                                              │
│      validated = _validate_todo_list(todo_list)             │
│                                                              │
│      # 返回 Command 更新 State                              │
│      return Command(                                         │
│          update={                                            │
│              "todo_list": validated,                         │
│              "messages": [ToolMessage(...)]                  │
│          }                                                   │
│      )                                                       │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑤ ToolNode 处理 Command，返回状态更新                      │
│                                                              │
│  ToolNode 返回：                                             │
│    {                                                         │
│      "todo_list": validated_tasks,                           │
│      "messages": [ToolMessage("任务列表已更新")]            │
│    }                                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑥ LangGraph 更新 AgentState                                │
│                                                              │
│  state.todo_list = [                                         │
│    {"id": "abc123", "name": "创建main.py", "status": "pending"},     │
│    {"id": "def456", "name": "创建test.py", "status": "pending"},     │
│    {"id": "ghi789", "name": "创建config.py", "status": "pending"}    │
│  ]                                                           │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑦ 回到 Agent 节点                                          │
└─────────────────────────────────────────────────────────────┘
```

#### 第3轮：调用 todo_read 查看任务

```
┌─────────────────────────────────────────────────────────────┐
│  ① Agent节点：LLM决定查看任务列表                           │
│                                                              │
│  LLM看到 ToolMessage："任务列表已更新"                       │
│    ↓                                                         │
│  LLM决定：需要查看任务列表，决定下一步                      │
│    ↓                                                         │
│  生成 tool_call：                                            │
│    {                                                         │
│      "name": "todo_read",                                    │
│      "arguments": {}  # 无参数，state 自动注入              │
│    }                                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ② ToolNode 执行 todo_read                                  │
│                                                              │
│  ③ ToolNode 检测到 state 参数有 InjectedState 注解          │
│  ④ 自动注入当前 state（包含最新的 todo_list）               │
│  ⑤ 调用 todo_read(state=current_state)                      │
│  ⑥ todo_read 读取 state.todo_list 并格式化                  │
│  ⑦ 返回格式化的任务列表字符串：                             │
│     "当前任务列表:                                           │
│      ⏳ 待处理:                                              │
│        [abc123] 创建main.py                                  │
│        [def456] 创建test.py                                  │
│        [ghi789] 创建config.py                                │
│      总计: 3 个任务"                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑧ 回到 Agent 节点                                          │
└─────────────────────────────────────────────────────────────┘
```

#### 第4轮：开始第一个任务

```
┌─────────────────────────────────────────────────────────────┐
│  ① Agent节点：LLM决定开始第一个任务                         │
│                                                              │
│  LLM看到格式化的任务列表                                    │
│    ↓                                                         │
│  LLM决定：开始第一个任务，标记为 in_progress                │
│    ↓                                                         │
│  生成 tool_call：                                            │
│    {                                                         │
│      "name": "todo_write",                                   │
│      "arguments": {                                          │
│        "todo_list": [                                        │
│          {"id": "abc123", "name": "创建main.py",            │
│           "status": "in_progress"},  # ⚠️ 标记为进行中      │
│          {"id": "def456", "name": "创建test.py",            │
│           "status": "pending"},                              │
│          {"id": "ghi789", "name": "创建config.py",          │
│           "status": "pending"}                               │
│        ]                                                     │
│      }                                                       │
│    }                                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ② ToolNode 执行 todo_write（更新状态为 in_progress）       │
│                                                              │
│  state.todo_list[0].status = "in_progress"                  │
│  state.todo_list[0].start_time = "2025-10-30T10:15:30"      │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ③ 回到 Agent 节点                                          │
└─────────────────────────────────────────────────────────────┘
```

#### 第5轮：执行实际工具

```
┌─────────────────────────────────────────────────────────────┐
│  ④ Agent节点：LLM调用实际工具完成任务                       │
│                                                              │
│  LLM看到："任务列表已更新"                                  │
│    ↓                                                         │
│  LLM决定：执行 create_file 创建文件                         │
│    ↓                                                         │
│  生成 tool_call：                                            │
│    {                                                         │
│      "name": "create_file",                                  │
│      "arguments": {                                          │
│        "filename": "main.py",                                │
│        "content": "print('Hello World')"                     │
│      }                                                       │
│    }                                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑤ ToolNode 执行 create_file                                │
│                                                              │
│  返回："✅ 已创建文件 main.py"                               │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑥ 回到 Agent 节点                                          │
└─────────────────────────────────────────────────────────────┘
```

#### 第6轮：标记任务完成

```
┌─────────────────────────────────────────────────────────────┐
│  ⑦ Agent节点：LLM标记任务完成                               │
│                                                              │
│  LLM看到："✅ 已创建文件 main.py"                            │
│    ↓                                                         │
│  LLM决定：第一个任务完成，标记为 completed                  │
│    ↓                                                         │
│  生成 tool_call：                                            │
│    {                                                         │
│      "name": "todo_write",                                   │
│      "arguments": {                                          │
│        "todo_list": [                                        │
│          {"id": "abc123", "name": "创建main.py",            │
│           "status": "completed"},  # ⚠️ 标记为已完成        │
│          {"id": "def456", "name": "创建test.py",            │
│           "status": "pending"},                              │
│          {"id": "ghi789", "name": "创建config.py",          │
│           "status": "pending"}                               │
│        ]                                                     │
│      }                                                       │
│    }                                                         │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑧ ToolNode 执行 todo_write（更新状态为 completed）         │
│                                                              │
│  state.todo_list[0].status = "completed"                    │
│  state.todo_list[0].end_time = "2025-10-30T10:16:45"        │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  ⑨ 循环继续处理下一个任务...                                │
│                                                              │
│  - 再次调用 todo_read 查看剩余任务                          │
│  - 标记第二个任务为 in_progress                             │
│  - 调用 create_file 创建 test.py                            │
│  - 标记第二个任务为 completed                               │
│  - ...重复直到所有任务完成                                  │
└─────────────────────────────────────────────────────────────┘
                    ↓
┌─────────────────────────────────────────────────────────────┐
│  最终：LLM 生成最终回答（无 tool_calls）→ END               │
└─────────────────────────────────────────────────────────────┘
```

### 🔑 关键要点

1. **任务清单由 LLM 在 Agent 节点自主生成**（不是硬编码）
2. **todo_read 频繁调用**：LLM 在执行任务前后都会查看任务列表
3. **ToolNode 自动注入 InjectedState 参数**（LLM 无需传递）
4. **todo_write 返回 Command 对象更新 State**（不是直接修改）
5. **ToolNode 自动处理 Command 返回值**（不需要包装器）
6. **完整的任务生命周期**：pending → in_progress → completed
7. **LLM 自主决策**：何时查看任务、何时开始、何时完成

### 📊 典型的执行模式

```
创建任务列表 (todo_write)
    ↓
查看任务 (todo_read) ←──────────┐
    ↓                           │
标记为 in_progress (todo_write) │
    ↓                           │
执行实际工具 (create_file)      │
    ↓                           │
标记为 completed (todo_write)   │
    ↓                           │
还有未完成的任务？──是──────────┘
    │
    否
    ↓
生成最终回答 → END
```

## 12. 测试：复杂任务自动分解

In [ ]:
print("=" * 80)
print(" 测试1: 复杂任务自动分解和跟踪")
print("=" * 80)
print()

result = graph.invoke({
    "messages": [HumanMessage(
        content="""请帮我完成以下任务：
1. 创建一个Python主文件 main.py，内容是一个简单的Hello World程序
2. 创建一个测试文件 test_main.py
3. 运行测试确保一切正常

将上面两个python文件放到 ./demo/ 目录中。
请使用任务列表跟踪进度。"""
    )],
    "todo_list": []  # 确保从空列表开始
}, {"recursion_limit": 100})

print("\n" + "=" * 80)
print(" 任务执行完成！")
print("=" * 80)

### 查看最终任务列表

In [ ]:
print("\n最终任务列表：")
print("=" * 80)

for i, task in enumerate(result["todo_list"], 1):
    status_emoji = {
        "pending": "⏳",
        "in_progress": "🔄",
        "completed": "✅",
        "failed": "❌"
    }
    emoji = status_emoji.get(task.status, "")
    
    print(f"{i}. {emoji} [{task.status}] {task.name}")
    print(f"   描述: {task.desc}")
    
    if task.start_time:
        print(f"   开始: {task.start_time}")
    if task.end_time:
        print(f"   结束: {task.end_time}")
    
    print()

### 查看最终回答

In [ ]:
print("\n最终回答：")
print("=" * 80)
final_message = result["messages"][-1]
print(final_message.content)

## 13. 测试：简单任务（不使用TodoList）

In [ ]:
print("=" * 80)
print(" 测试2: 简单任务（不使用TodoList）")
print("=" * 80)
print()

result2 = graph.invoke({
    "messages": [HumanMessage(content="创建一个名为 hello.txt 的文件")],
    "todo_list": []
})

print("最终回答:")
print(result2["messages"][-1].content)
print(f"\n任务列表是否为空: {len(result2['todo_list']) == 0}")

## 16. 练习题

### 练习1: 添加任务优先级

扩展 `TodoItem`，添加 `priority` 字段（high/medium/low）。

提示：
1. 在 `TodoItem` 中添加 `priority: str = "medium"`
2. 修改 `format_todo_list` 显示优先级
3. 更新系统提示词，指导 LLM 设置优先级

### 练习2: 任务依赖

添加 `depends_on` 字段，实现任务依赖关系。

提示：
1. 添加 `depends_on: Optional[List[str]] = None`（存储依赖的任务ID）
2. 在 `_validate_todo_list` 中检查依赖是否已完成
3. 更新提示词，教 LLM 如何处理依赖

### 练习3: 任务历史

在 State 中添加 `task_history`，记录所有已完成的任务。

提示：
1. 在 `AgentState` 中添加 `task_history: List[TodoItem] = Field(default_factory=list)`
2. 在 `todo_write` 中，当任务标记为 completed 时，追加到 history
3. 创建 `todo_history` 工具查看历史

### 思考题

1. **为什么 TodoWrite 返回 Command 而不是字符串？**
   - 提示：思考如何同时更新 todo_list 和 messages

2. **如何确保 LLM 按顺序执行任务？**
   - 提示：系统提示词中的哪些部分起作用？

3. **提示词中哪些部分最重要？**
   - 提示：触发条件 vs 示例 vs 规则

4. **能否让工具直接修改 state 而不用 Command？为什么不推荐？**
   - 提示：考虑 LangGraph 的状态管理机制

## 17. 提示词工程心得

从 TodoList 的实现中，我们学到：

### 1. 明确指令
- ✅ "主动且频繁地使用此工具"
- ✅ "你必须使用 todo_write 创建任务列表"
- ❌ "可以使用此工具"

### 2. 具体场景
- ✅ "当任务需要3个或更多步骤时"
- ✅ "收到复杂的多步骤任务时 - 立即分解为子任务"
- ❌ "当任务复杂时"

### 3. Few-shot 示例
提供具体例子比长篇描述更有效：
```
用户: "创建一个Python项目，包含主文件、测试文件，并运行测试"

你的行动:
1. 使用 todo_write 创建任务列表
2. 执行第一个任务前，标记为 in_progress
3. 完成后，标记为 completed
...
```

### 4. 强调重要性
使用加粗、重复、示例来强调关键规则：
- "主动使用此工具"
- "频繁使用 todo_read"
- "实时更新任务状态"


## 18. 相关资源

- [独立脚本版本](./04_todo_task_management.py)
- [完整实现：claude_code_demo](./claude_code_demo/)

## 19. 下一步

在下一章（第5章），我们将学习：
- Token 监控机制（反向查找优化）
- 上下文压缩（8段式压缩提示词）
- 压缩节点设计
- 消息历史管理

---

**🎉 恭喜！你已经掌握了 Claude Code 最核心的任务管理功能！**